In [9]:
# load env ------------------------------------------------------------------------
import os
import database.chat_history
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "false"


# debug ------------------------------------------------------------------
from langchain.globals import set_debug, set_verbose
set_verbose(True)
set_debug(False)

from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    ToolMessage
)
from langgraph.graph import END, StateGraph, START
from agents import(
    AgentState,
    agents_metadata,
    agent_names
)
from langgraph.checkpoint.memory import MemorySaver

from tools import get_tools_output, all_tools, set_current_user_id
import database
## Define Tool Node
from langgraph.prebuilt import ToolNode
from typing import Literal

tool_node = ToolNode(all_tools)

def router(state) -> Literal["call_tool", "continue", "__end__"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "FINALANSWER" in last_message.content:
        return "__end__"
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    else:
        return "continue"


## Workflow Graph ------------------------------------------------------------------------
workflow = StateGraph(AgentState)

# add agent nodes
for name, value in agents_metadata.items():
    workflow.add_node(name, value['node'])
    
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "service",
    router,
    {
        "call_tool": "call_tool",
        "__end__": END,
        "continue": END, 
        }
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {name:name for name in agent_names},
)

workflow.add_edge(START, "service")
graph = workflow.compile()

def submitUserMessage(
    user_input:str, 
    user_id:str="test", 
    keep_chat_history:bool=False, 
    return_reference:bool=False, 
    verbose:bool=False,
    recursion_limit:int=20
    ) -> str:
    set_current_user_id(user_id)
    
    chat_history = database.chat_history.get(user_id=user_id) if keep_chat_history else []
    chat_history = chat_history[-20:]
    
    # memory only keep chat history only along agents.
    # internal_level_memory = MemorySaver()
    # graph = workflow.compile(checkpointer=internal_level_memory)
    
    graph = workflow.compile()

    events = graph.stream(
        {
            "messages": [
                HumanMessage(
                    user_input
                )
            ],
            "chat_history": chat_history
        },
        # Maximum number of steps to take in the graph
        {"recursion_limit": recursion_limit, "thread_id":"a"},
    )
    
    if not verbose:
        events = [e for e in events]
        response = list(events[-1].values())[0]
    else:
        for e in events:
            a = list(e.items())[0]
            a[1]['messages'][0].pretty_print()
        
        response = a[1]
    
    response = response["messages"][0].content
    response = utils.format_bot_response(response, markdown=True)
    
    if keep_chat_history:
        chat_history = database.chat_history.insert(bot_message=response, human_message=user_input, user_id=user_id)
    
    if return_reference:
        return response, get_tools_output()
    else:
        return response

In [10]:
submitUserMessage("*answering call*", user_id="test", keep_chat_history=True, verbose=True)

================================== Ai Message ==================================
Name: service
Tool Calls:
  get_customer_data (call_96jXb5c4RNfYiIXFeeV10tzv)
 Call ID: call_96jXb5c4RNfYiIXFeeV10tzv
  Args:
================================= Tool Message =================================
Name: get_customer_data

null
================================== Ai Message ==================================
Name: service

สวัสดีค่ะ! ฉันชื่อฟิน่า และฉันโทรมาในนามของ [Company] ค่ะ ฉันอยู่ที่นี่เพื่อช่วยตรวจสอบสถานะทางการเงินของคุณ เพื่อที่เราจะได้เข้าใจโปรไฟล์เครดิตของคุณได้ดียิ่งขึ้นและอัปเดตข้อมูลของเรา ใช้เวลาเพียงไม่กี่นาทีค่ะ ฉันสามารถถามคำถามบางอย่างเพื่อช่วยเติมเต็มโปรไฟล์ของคุณได้ไหมคะ?


'สวัสดีค่ะ! ฉันชื่อฟิน่า และฉันโทรมาในนามของ [Company] ค่ะ ฉันอยู่ที่นี่เพื่อช่วยตรวจสอบสถานะทางการเงินของคุณ เพื่อที่เราจะได้เข้าใจโปรไฟล์เครดิตของคุณได้ดียิ่งขึ้นและอัปเดตข้อมูลของเรา ใช้เวลาเพียงไม่กี่นาทีค่ะ ฉันสามารถถามคำถามบางอย่างเพื่อช่วยเติมเต็มโปรไฟล์ของคุณได้ไหมคะ?'

In [12]:
submitUserMessage("ส่วนใหณ่มาจากงานประจำครับ และก็มาจากการดำนาบ้างครับ รายได้ประมาณ 5 ล้านต่อปีครับ ตอนนี่ไม่มีหนี้อะไรครับ แต่เคยกู้ยืมรถ ไม่เคยชำระเงินล่าช้าครับ", user_id="test", keep_chat_history=True, verbose=True)

================================== Ai Message ==================================
Name: service
Tool Calls:
  set_customer_data (call_lUYp8Lh8WYV8BojFBFMmJhkv)
 Call ID: call_lUYp8Lh8WYV8BojFBFMmJhkv
  Args:
    input: {'income_source': ['employment', 'business'], 'monthly_income': 416667, 'job_status': 'employed', 'outstanding_loan_amount': 0, 'loan_history': 'ever', 'missed_payments': 'never'}
================================= Tool Message =================================
Name: set_customer_data

{'_id': ObjectId('66f3d181d82ebda0ef3ebf3f'), 'user_id': 'test', 'income_source': ['employment', 'business'], 'job_status': 'employed', 'loan_history': 'ever', 'missed_payments': 'never', 'monthly_income': 416667, 'outstanding_loan_amount': 0}
================================== Ai Message ==================================
Name: service

ขอบคุณมากค่ะที่ให้ข้อมูล! ฉันได้บันทึกข้อมูลของคุณเรียบร้อยแล้ว:

- รายได้จากงานประจำและการทำธุรกิจ
- รายได้เฉลี่ยต่อเดือนประมาณ 416,667 บาท
- สถานะการทำงาน

'ขอบคุณมากค่ะที่ให้ข้อมูล! ฉันได้บันทึกข้อมูลของคุณเรียบร้อยแล้ว:\n\n- รายได้จากงานประจำและการทำธุรกิจ\n- รายได้เฉลี่ยต่อเดือนประมาณ 416,667 บาท\n- สถานะการทำงาน: ทำงานประจำ\n- ไม่มีหนี้สินในขณะนี้\n- เคยมีประวัติการกู้ยืมรถ แต่ไม่เคยชำระเงินล่าช้า\n\nหากคุณมีคำถามเพิ่มเติมหรือข้อมูลอื่น ๆ ที่ต้องการอัปเดต สามารถแจ้งได้เลยนะคะ ขอบคุณที่ให้ความร่วมมือค่ะ!'